In [1]:
!nvidia-smi

Sun Aug 27 06:47:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000001:00:00.0 Off |                    0 |
| N/A   32C    P0    42W / 300W |      4MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-13b-hf', trust_remote_code=True)

In [5]:
model_meso = AutoModelForCausalLM.from_pretrained(
    'mesolitica/llama-13b-hf-2048-fpf', 
    device_map="auto", 
    load_in_4bit=True,
)

[2023-08-27 06:47:58,792] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-27 06:47:59.161862: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 06:47:59.813193: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
model_base = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-13b-hf', 
    device_map="auto", 
    load_in_4bit=True,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

In [31]:
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False
    
stop_words = ["\n", "\nYou:", "You:"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze()[1:] for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [43]:
prompts = [
    'Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.',
    'Teruskan dialog di bawah. Tulis satu jawapan untuk watak "Assistant". Anda boleh bertanya kepadanya apa sahaja yang anda mahu dan ia akan melakukan yang terbaik untuk memberi anda maklumat yang tepat dan berkaitan.'
]

In [44]:
questions = [
    'siapa perdana menteri malaysia',
    'KWSP tu apa',
    'awat rafizi introduced vending machine?',
    'kerajaan madani banyak membantu tak?'
]

In [45]:
max_new_tokens = 500
temperature = 0.8
top_p = 0.95
top_k = 20
num_beams = 1
do_sample = True
repetition_penalty = 1.15

In [46]:
for p in prompts:
    for q in questions:
        query = f"""
{p}
You: {q}
Assistant:
        """
        query = query.strip()
        inputs = tokenizer(query, return_tensors='pt')
        input_ids = inputs['input_ids'].to(model_meso.device)

        with torch.no_grad():
            generation_output = model_meso.generate(
                input_ids=input_ids,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                num_beams=num_beams,
                do_sample=do_sample,
                stopping_criteria=stopping_criteria,
                repetition_penalty=repetition_penalty,
        )
        t = tokenizer.decode(generation_output.sequences[0])
        t = t.split('Assistant: ')[1]
        t = t.split('You:')[0]
        print(query)
        print(t)
        print()
        

Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: siapa perdana menteri malaysia
Assistant:
Tun Dr Mahathir bin Mohamad adalah Perdana Menteri Malaysia ke-4 dan berkhidmat dari 1981 hingga 2003. Beliau telah mengetuai Barisan Nasional (BN) untuk memenangi pilihan raya umum dengan majoriti besar sejak tiga kali berturut-turut iaitu pada tahun 1995, 1999 dan 2004. Beliau juga merupakan Perdana Menteri pertama yang dipilih oleh Majlis Tindakan Ekonomi Negara (NEAC), satu agensi kerajaan baru yang ditubuhkan di bawah pentadbirannya. Beliau berjaya mengurus Malaysia sebagai salah sebuah negara paling teratur di Asia. Beliau telah dilantik sebagai Pengerusi Kelab Perkembangan Pemimpin Dunia anjuran World Economic Forum dan dinobatkan sebagai 'The Asian of The Year' oleh majalah Newsweek atas kejayaan beliau dalam menstabilkan ekonomi Malaysia serta 

In [47]:
for p in prompts:
    for q in questions:
        query = f"""
{p}
You: {q}
Assistant:
        """
        query = query.strip()
        inputs = tokenizer(query, return_tensors='pt')
        input_ids = inputs['input_ids'].to(model_base.device)

        with torch.no_grad():
            generation_output = model_base.generate(
                input_ids=input_ids,
                return_dict_in_generate=True,
                output_scores=True,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                num_beams=num_beams,
                do_sample=do_sample,
                stopping_criteria=stopping_criteria,
                repetition_penalty=repetition_penalty,
        )
        t = tokenizer.decode(generation_output.sequences[0])
        t = t.split('Assistant: ')[1]
        t = t.split('You:')[0]
        print(query)
        print(t)
        print()

Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: siapa perdana menteri malaysia
Assistant:
The Prime Minister of Malaysia is Tun Dr Mahathir Mohamad, who took office in May 2018.


Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relevant information.
You: KWSP tu apa
Assistant:
I am an AI-powered virtual assistant created by KWSP, one of Malaysia's leading technology companies specializing in data analysis and processing solutions for businesses. My role is to provide assistance to customers regarding their queries related to our products and services.


Continue the chat dialogue below. Write a single reply for the character "Assistant". You can ask it anything you want and it will do its best to give you accurate and relev